In [1]:
import argparse
import random
import numpy as np
import os
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
from torchvision import models
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Subset

from covidxdataset import COVIDxDataset
import util as util
from train import train, validation

In [2]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

In [8]:
seed = 20
seed_everything(seed)

num_classes = 2
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
model.fc = nn.Linear(num_ftrs, num_classes)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

batch_size = 128
train_params = {'batch_size': batch_size,
                'shuffle': True,
                'num_workers': 3}

test_params = {'batch_size': batch_size,
               'shuffle': False,
               'num_workers': 1}

optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

dataset_name = 'COVIDx'
if dataset_name == 'COVIDx':
    train_dataset = COVIDxDataset(mode='train', n_classes=num_classes, dim=(224, 224))
    val_dataset = COVIDxDataset(mode='valid', n_classes=num_classes, dim=(224, 224))
    test_dataset = COVIDxDataset(mode='test', n_classes=num_classes, dim=(224, 224))
    
    train_loader = DataLoader(train_dataset, **train_params)
    val_loader = DataLoader(val_dataset, **test_params)
    test_loader = DataLoader(test_dataset, **test_params)

elif dataset_name == 'COVID_CT':
    train_dataset = CovidCTDataset('train',root_dir='./data/covid_ct_dataset',
                                  txt_COVID='./data/covid_ct_dataset/trainCT_COVID.txt',
                                  txt_NonCOVID='./data/covid_ct_dataset/trainCT_NonCOVID.txt')
    val_dataset = CovidCTDataset('val',root_dir='./data/covid_ct_dataset',
                                txt_COVID='./data/covid_ct_dataset/valCT_COVID.txt',
                                txt_NonCOVID='./data/covid_ct_dataset/valCT_NonCOVID.txt')
    test_dataset = CovidCTDataset('test',root_dir='./data/covid_ct_dataset',
                                 txt_COVID='./data/covid_ct_dataset/testCT_COVID.txt',
                                 txt_NonCOVID='./data/covid_ct_dataset/testCT_NonCOVID.txt')

    train_loader = DataLoader(train_dataset, **train_params)
    val_loader = DataLoader(val_dataset, **test_params)
    test_loader = DataLoader(test_dataset, **test_params)

# print(model)
num_epochs = 10
best_pred_loss = 1000.0
lr_sch = ReduceLROnPlateau(optimizer, factor=0.5, patience=2, min_lr=1e-6, verbose=True)
# lr_sch = ExponentialLR(optimizer, gamma=0.975)

for epoch in range(1, num_epochs + 1):
    train(device, batch_size, model, train_loader, optimizer, epoch, None)
    val_metrics, confusion_matrix = validation(device, batch_size, num_classes, model, val_loader, epoch, None)

#     best_pred_loss = util.save_model(model, optimizer, args, val_metrics, epoch, best_pred_loss, confusion_matrix)
    
    lr_sch.step(val_metrics.avg('loss'))
#     lr_sch.step()

train examples =  12562
valid examples =  1396
test examples =  1579
('loss', 'correct', 'total', 'accuracy')
Epoch: 1	Sample:    1/12672	Loss:0.7168	Accuracy:0.47
Training
 SUMMARY EPOCH: 1	Sample:12545/12545	Loss:0.3191	Accuracy:0.93

('loss', 'correct', 'total', 'accuracy')
Validation
 SUMMARY EPOCH: 1	Sample: 1281/ 1281	Loss:0.1776	Accuracy:0.97

Confusion Matrix
[[1327.   17.]
 [  20.   32.]]
('loss', 'correct', 'total', 'accuracy')
Epoch: 2	Sample:    1/12672	Loss:0.1065	Accuracy:0.98
Training
 SUMMARY EPOCH: 2	Sample:12545/12545	Loss:0.0805	Accuracy:0.98

('loss', 'correct', 'total', 'accuracy')
Validation
 SUMMARY EPOCH: 2	Sample: 1281/ 1281	Loss:0.1144	Accuracy:0.97

Confusion Matrix
[[1318.   26.]
 [  15.   37.]]
('loss', 'correct', 'total', 'accuracy')
Epoch: 3	Sample:    1/12672	Loss:0.0604	Accuracy:0.98
Training
 SUMMARY EPOCH: 3	Sample:12545/12545	Loss:0.0505	Accuracy:0.99

('loss', 'correct', 'total', 'accuracy')
Validation
 SUMMARY EPOCH: 3	Sample: 1281/ 1281	Loss:0.100

In [9]:
test_metrics, confusion_matrix = validation(device, batch_size, num_classes, model, test_loader, epoch, None)

('loss', 'correct', 'total', 'accuracy')
Validation
 SUMMARY EPOCH:10	Sample: 1537/ 1537	Loss:0.1123	Accuracy:0.96

Confusion Matrix
[[1435.   44.]
 [  19.   81.]]
